### Installing packages


In [22]:
install.packages("tidyverse")
install.packages("cowplot")
install.packages("digest")
install.packages("infer")
install.packages("repr")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [23]:
library(cowplot)
library(digest)
library(infer)
library(repr)
library(tidyverse)

In [1]:
1 + 2

[1] 3